<a href="https://colab.research.google.com/github/gjrkdgh0772/Finance/blob/main/lec07_%EC%84%A0%EB%AC%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6><b>Lec07. 파생상품-선물(Future)

* <b>파생상품</b><br>
   기초자산의 가치 변동에 따라 가격이 결정되는 금융상품<br>
* <b>선물거래</b><br>  
   기초자산을 미래의 일정 시점에 미리 정한 가격으로 인도, 인수할 것을 '약속'하는 거래  <br>
* <b>옵션거래</b><br>  
   미리 정한 가격으로 사거나 팔 수 있는 '권리'를 거래하는 것<br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)


# Data Load

In [ ]:
df = pd.read_excel("./dataset/lec07_future/선물이론가산출1.xlsx", '선물', usecols=['일자','지수','선물이론가(1분기)','선물이론가(2분기)'])
df.head()

,일자,지수,선물이론가(1분기),선물이론가(2분기)
0,1,100.000000,100.184694,100.372717
1,2,100.060680,100.242502,100.430633
2,3,100.235372,100.414524,100.602977
3,4,99.724548,99.899813,100.087301
4,5,98.970884,99.141874,99.327939


# 선물이론가 
* 이론가 = 현물가 + 순보유비용(만기 까지의 이자비,보관비 등)
* 𝑭 =𝑺𝒆^(𝒓−𝒅)𝒕   

In [ ]:
만기_1분기 = 63
만기_2분기 = 126

In [ ]:
# def my_feture_price(df, 만기일자, 이자율=2.25, 배당률=1.50) :
#     잔존만기 = (만기_1분기 - df['일자'])/252
#     선물이론가 = 현물가 * np.exp((이자율-배당률) * 잔존만기)
#     return 선물이론가

def my_feture_price(df, 만일일자,  r=0.0225, d=0.0150) :
    t = (만일일자 - df['일자'])/252
    S = df['지수']
    price = S * np.exp((r-d) * t)
    return price

In [ ]:
df['이론가1'] = df.apply(lambda x : my_feture_price(x, 53), axis=1)
df['이론가2'] = df.apply(lambda x : my_feture_price(x, 126), axis=1)
df.head()

,일자,지수,선물이론가(1분기),선물이론가(2분기),이론가1,이론가2
0,1,100.000000,100.184694,100.372717,100.154882,100.372717
1,2,100.060680,100.242502,100.430633,100.212673,100.430633
2,3,100.235372,100.414524,100.602977,100.384643,100.602977
3,4,99.724548,99.899813,100.087301,99.870086,100.087301
4,5,98.970884,99.141874,99.327939,99.112372,99.327939


# [미션] WTI 유가 선문옵션 만기에 따른 수익률 비교

## Data Load

In [ ]:
df1 = pd.read_excel("./dataset/lec07_future/WTI.xlsx", 'Spot'  , index_col='Date')
df2 = pd.read_excel("./dataset/lec07_future/WTI.xlsx", 'Front' , index_col='Date')
df3 = pd.read_excel("./dataset/lec07_future/WTI.xlsx", 'Second', index_col='Date')
df4 = pd.read_excel("./dataset/lec07_future/WTI.xlsx", 'Expiry', usecols=['Last Trade'])

In [ ]:
df = pd.concat([df1, df2, df3], axis=1)
df.head()

,Spot,Front,Second
Date,,,
2017-12-28,59.84,59.84,59.87
2017-12-27,59.64,59.64,59.69
2017-12-26,59.97,59.97,60.00
2017-12-22,58.42,58.47,58.54
2017-12-21,58.33,58.36,58.40


In [ ]:
df.isna().sum()

Spot      0
Front     0
Second    0
dtype: int64

In [ ]:
df4.tail()

,Last Trade
288,11/20/24
289,05/20/25
290,11/20/25
291,05/19/26
292,11/20/26


## 만기일여부 (1,0)

In [ ]:
df44 = pd.to_datetime(df4['Last Trade'])
df44.head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [ ]:
df.head()

,Spot,Front,Second
Date,,,
2017-12-28,59.84,59.84,59.87
2017-12-27,59.64,59.64,59.69
2017-12-26,59.97,59.97,60.00
2017-12-22,58.42,58.47,58.54
2017-12-21,58.33,58.36,58.40


In [ ]:
df44.values[2], pd.to_datetime(df44.values)[:2]

(numpy.datetime64('2000-03-21T00:00:00.000000000'),
 DatetimeIndex(['2000-01-20', '2000-02-22'], dtype='datetime64[ns]', freq=None))

In [ ]:
df['Expiry'] = 0
for idx in df.index:
    if idx in pd.to_datetime(df44.values):
        df.loc[idx,'Expiry'] = 1

In [ ]:
df['Expiry'].value_counts()

0    1918
1      96
Name: Expiry, dtype: int64

In [ ]:
df.head(10)

,Spot,Front,Second,Expiry
Date,,,,
2017-12-28,59.84,59.84,59.87,0
2017-12-27,59.64,59.64,59.69,0
2017-12-26,59.97,59.97,60.00,0
2017-12-22,58.42,58.47,58.54,0
2017-12-21,58.33,58.36,58.40,0
2017-12-20,58.02,58.09,58.13,0
2017-12-19,57.46,57.46,57.56,1
2017-12-18,57.16,57.16,57.22,0
2017-12-15,57.30,57.30,57.33,0


In [ ]:
df = df.sort_index(ascending=True)
df.head()

,Spot,Front,Second,Expiry
Date,,,,
2010-01-04,81.51,81.51,82.12,0
2010-01-05,81.77,81.77,82.41,0
2010-01-06,83.18,83.18,83.75,0
2010-01-07,82.66,82.66,83.19,0
2010-01-08,82.75,82.75,83.30,0


# 현물투자 vs 선물투자 수익률 비교

* Front : 최근월물 : 만기를 앞두고 있는 선물
* Second : 차근월물 : 다음 만기를 앞두고 있는 선물

In [ ]:
df.index.min(), df.index.max()

(Timestamp('2010-01-04 00:00:00'), Timestamp('2017-12-28 00:00:00'))

## 현물에 7년간 투자했을 경우

In [ ]:
df.loc[ df.index.max(),'Spot'] - df.loc[df.index.min(), 'Spot']

-21.67

## 선물에 7년간 투자했을 경우 -> 롤오버
* 롤오버 : 만기시 차근월물로 교체

In [ ]:
df['rollover'] = 0
df.head(2)

,Spot,Front,Second,Expiry,rollover
Date,,,,,
2010-01-04,81.51,81.51,82.12,0,0
2010-01-05,81.77,81.77,82.41,0,0


* by 최상현

In [ ]:
df['rollover']=df[df['Expiry']==1]['Second']-df[df['Expiry']==1]['Front']
df['rollover'] = df['rollover'].fillna(0)

* index를 이용한 변경

In [ ]:
idx = df[df['Expiry'] ==1].index
print(idx[:5])

DatetimeIndex(['2010-01-20', '2010-02-22', '2010-03-22', '2010-04-20',
               '2010-05-20'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [ ]:
for i in idx:
    df.loc[i, 'rollover'] = df.loc[i, 'Second'] - df.loc[i, 'Front']

In [ ]:
df.head(15)

,Spot,Front,Second,Expiry,rollover
Date,,,,,
2010-01-04,81.51,81.51,82.12,0,0.00
2010-01-05,81.77,81.77,82.41,0,0.00
2010-01-06,83.18,83.18,83.75,0,0.00
2010-01-07,82.66,82.66,83.19,0,0.00
2010-01-08,82.75,82.75,83.30,0,0.00
2010-01-11,82.52,82.52,83.01,0,0.00
2010-01-12,80.79,80.79,81.17,0,0.00
2010-01-13,79.65,79.65,80.04,0,0.00
2010-01-14,79.39,79.39,79.88,0,0.00


### 선물손익 = 현재가 - 최초가 - 롤오버비용

In [ ]:
tot_rollover = df['rollover'].cumsum()[-1]
tot_rollover

40.680000000000035

In [ ]:
df.loc[ df.index.max(),'Spot'] - df.loc[df.index.min(), 'Spot'] - tot_rollover

-62.35000000000004